## Data Preparation
Join lcbo_listing and lcbo_product as one dataframe, clean and tidy columns and preprocess text features

In [15]:
import numpy as np
import pandas as pd
from unidecode import unidecode

In [16]:
import sys
sys.path.insert(0, '../modules')
from helpers import clean_text

### Load and Merge Dataset

In [65]:
listing = pd.read_csv('../data/raw/lcbo_listing_20190626.csv')
product = pd.read_csv('../data/raw/lcbo_product_20190626.csv')

# merge listing and product
lcbo = pd.merge(listing, product)

# subset dataset with product price less than C$500 (scope of the project)
lcbo = lcbo[lcbo.price < 500]

# check dataset shape and missing values
print(lcbo.shape)
lcbo.isnull().sum()

(9307, 7)


name              0
price             0
prod_url          0
category        195
description    1025
details           0
sku               0
dtype: int64

### Initial Data Cleaning

In [66]:
# flatten 'details' column to separate columns
lcbo['details'] = lcbo['details'].map(eval)
lcbo = pd.concat([lcbo.drop(columns=['details']), lcbo['details'].apply(pd.Series)], axis=1)

# clean column names
lcbo.columns = (lcbo.columns.str.replace(':', '')
                            .str.replace(' |/', '_')
                            .str.lower())

# extract country and region info from made_in
lcbo['made_in'] = lcbo.made_in.str.strip().str.split(', ')
lcbo['country'] = np.where(lcbo.made_in.str.len() == 2, lcbo.made_in.str[1], lcbo.made_in.str[0])
lcbo['region'] = np.where(lcbo.made_in.str.len() == 2, lcbo.made_in.str[0], np.nan)

# extract numeric values from columns
lcbo['bottle_size'] = lcbo.bottle_size.str.extract('(\d+)').astype(int)
lcbo['sugar_content'] = lcbo.sugar_content.str.extract('(\d+)').astype(float)
lcbo['alcohol_vol'] = lcbo.alcohol_vol.str.replace('%', '').astype(float)

# extract score info from descriptions
lcbo['score'] = lcbo.description.str.extract('(?i)\Wscore\W.*?(\d{2})').astype(float)
lcbo['score'] = np.where(lcbo.score < 60, np.nan, lcbo.score)

# extract wine type from product url
lcbo['wine_type'] = np.where(lcbo.prod_url.str.contains('red-wine-14001'), 'red',
                    np.where(lcbo.prod_url.str.contains('white-wine-14002'), 'white',
                    np.where(lcbo.prod_url.str.contains('ros%C3%A9-wine-14003'), 'rose',
                             'sparkling')))

# separate SKU# and group (lcbo or vintage)
lcbo['group'] = (lcbo.sku.str.split(':')
                         .str[0]
                         .str.replace('#', '')
                         .str.strip())
lcbo['sku'] = (lcbo.sku.str.split(':')
                       .str[1]
                       .str.strip())

### Preprocess Text Columns

In [67]:
# replace field that is entirely space or empty with NaN
lcbo.replace(r'^\s*$', np.nan, regex=True, inplace=True)

#### Preprocess varietal column

In [ ]:
# fill missing varietal info with category
lcbo['varietal'].fillna(lcbo['category'], inplace=True)

# clean text in varietal column
lcbo['varietal'] = lcbo.varietal.apply(lambda x: x if pd.isnull(x) else unidecode(x))
replace_list = ['\(.+?\)', ' blend', ' table wines', ' wines', '^named.*', ' red', ' white', 
                ' rose', '^other.*', '^regionaled.*', ' - other', ' - meritage']
lcbo['varietal'] = (lcbo.varietal.str.lower()
                                 .str.replace('|'.join(replace_list), '', regex=True)
                                 .str.replace('/ ', '/'))

# replace alternative names
lcbo.varietal.replace(['negro amaro', 'refosco dal peduncolo rosso', 'tokaj', 'aragones'],
                      ['negroamaro', 'refosco', 'tokaji', 'aragonez'], inplace=True)

# replace empty strings after preprocessing with NaN
lcbo['varietal'].replace(r'^\s*$', np.nan, regex=True, inplace=True)

# fill missing varietal info with wine type
lcbo['varietal'].fillna(lcbo['wine_type'], inplace=True)

#### Prepocess description column

In [ ]:
# fill missing description
lcbo['description'].fillna('na', inplace=True)

# clean text in description
lcbo['description'] = lcbo.description.apply(clean_text)

### Drop Columns

In [69]:
# drop columns that are no longer needed
drop_cols = ['prod_url', 'category', 'made_in', 'by', 'sweetness_descriptor', 'style',
             'release_date', 'this_is_a_vqa_wine', 'this_is_a_kosher_product.']
lcbo.drop(columns= drop_cols, inplace=True)

### Export Processed Dataset

In [70]:
lcbo.to_csv('../data/processed/lcbo.csv', index=False)